# Imports

In [266]:
# handle math and data
import numpy as np
import pandas as pd
import time
import math

# to plot nice figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# handle files
import os
import sys
import joblib

# hash table classes
from collections import Counter

# output
import tqdm

# deep learning
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
import tensorflow_hub as hub

# Setup

In [2]:
SEED = 69

K = keras.backend

AUTO = tf.data.AUTOTUNE

def reset_backend():
    K.clear_session()
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

# Learning

## Generate Shakespearean Text With Character RNN

### Get Data

In [3]:
DOWNLOAD_URL = "https://homl.info/shakespeare"
filepath = keras.utils.get_file("shakespeare.txt", DOWNLOAD_URL)

1115394/1115394 [==============================] - 0s 0us/step


In [4]:
filepath

'/Users/calvinhuang/.keras/datasets/shakespeare.txt'

In [7]:
with open(filepath) as f:
    shakespeare_text = f.read()

In [8]:
len(shakespeare_text)

1115394

Over 1 million characters.

Let's encode our text, basically char to indices.

In [9]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

Now we can use the tokenizer.

Wrap the text in a list, because the method by default will convert each text in the given `texts`. Which means if you just pass the string, it will convert each character to its own sequence and you will have a list of lists.

In [36]:
tokenizer.texts_to_sequences(["Hi! My name is Calvin!\n"])

[[7, 6, 31, 1, 15, 16, 1, 10, 5, 15, 2, 1, 6, 8, 1, 19, 5, 12, 26, 6, 10, 31, 11]]

In [37]:
tokenizer.sequences_to_texts([[7, 6, 31, 1, 15, 16, 1, 10, 5, 15, 2, 1, 6, 8, 1, 19, 5, 12, 26, 6, 10, 31, 11]])

['h i !   m y   n a m e   i s   c a l v i n ! \n']

In [40]:
tokenizer.word_index

{' ': 1,
 'e': 2,
 't': 3,
 'o': 4,
 'a': 5,
 'i': 6,
 'h': 7,
 's': 8,
 'r': 9,
 'n': 10,
 '\n': 11,
 'l': 12,
 'd': 13,
 'u': 14,
 'm': 15,
 'y': 16,
 'w': 17,
 ',': 18,
 'c': 19,
 'f': 20,
 'g': 21,
 'b': 22,
 'p': 23,
 ':': 24,
 'k': 25,
 'v': 26,
 '.': 27,
 "'": 28,
 ';': 29,
 '?': 30,
 '!': 31,
 '-': 32,
 'j': 33,
 'q': 34,
 'x': 35,
 'z': 36,
 '3': 37,
 '&': 38,
 '$': 39}

In [41]:
max_id = len(tokenizer.word_index)
max_id

39

In [42]:
dataset_size = tokenizer.document_count
dataset_size

1115394

So the tokenizer can be fit to text and it will create a word_index hashtable aka dict for each char or word. Then you can easily map texts to sequences and vice versa. Let's create our encoded data now.

Subtract by 1 so that our enocded indices will be from 0 to 38

In [48]:
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [49]:
encoded.shape

(1115394,)

In [50]:
np.unique(encoded)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38])

Now let's create our efficient datasets.

### Create Datasets

- Set train set to first 90% of text
- Create dataset with windows of 100 characters
    - Window size of 101 because target is next char
- Flat map with batch on windows
- Shuffle
- Batch datset
- Split input and target data
- Apply one hot encoding to inputs
- Prefetch

In [59]:
train_size = dataset_size * 90 // 100
train_size

1003854

In [65]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

Now let's get windows of data.

In [66]:
n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

This creates a dataset of window datasets (yes each window is a dataset). To get just one dataset and convert windows to tensors, we need to use `flat_map` and batch each window to it's own size.

In [68]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

Now let's shuffle our windows and then batch them.

In [69]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)

Let's split our data into input and target sequences.

Shape of our data: `(batch_size, 101)`

In [70]:
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

Now let's apply one hot to our inputs.

Shape of our data: `(input (batch_size, 100, 1), target (batch_size, 100, 1))`

In [72]:
dataset = dataset.map(lambda X_batch, Y_batch:
                          (tf.one_hot(X_batch, max_id), Y_batch))

Finally, let's prefetch.

In [74]:
dataset = dataset.prefetch(1)

### Model Arch.

Let's use two GRU layers with 128 units each for long term memory. Add 20% dropout to both inputs and hidden states to prevent unstable gradients since this model will run through 100 steps or 100 layers when unrolled (which is pretty deep). Finally, we have a timedistributed dense layer (which means it will output for every step, not just last output) that will output 39 probabilities for the 39 possible characters using a softmax activation.

- 2 GRU layers, 128 units, 20% dropout
- Dense layer, 39 units, softmax

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, 
                                                    activation="softmax"))
])

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="nadam",
              metrics=["accuracy"])

history = model.fit(dataset, epochs=10)

This model takes way too long to train, let's use a stateful model and just copy the weights over.

### Stateful Model

Stateful means copying the last hidden state of the previous iteration over to the new iteration instead of using 0s.

To do this we will need our batches to be consecutive, an easy way is to have batches of size 1. Also our windows will need to have a shift of n_steps (not n_steps + 1 since we only add 1 for the outputs).

In [80]:
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1) # no shuffle and batch of size 1
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:])) # batches are just one window
dataset = dataset.map(lambda X_batch, Y_batch: 
                      (tf.one_hot(X_batch, max_id), Y_batch)) # again, batches are just one window
dataset.prefetch(1)

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 39), dtype=tf.float32, name=None), TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>

But then we won't be able to take advantage of vectorization performance improvements.

So, let's do this:
- Split our data into 32 chunks in order
- Window each dataset
- Stack the datasets
    - Each element should be a (32, 101) tensor
    - The first element is the first 101 steps or first window of the 32 datasets
    - No need to batch now, since it's already in batch shape after stacking
- Split input and target
- Apply one hot 
- Prefetch

In [89]:
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
    dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
    dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_length))
    datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

Now let's train our model - we need to set stateful to True for our RNN layers. Also, we need to make sure to reset the state between epochs (because we don't want to have bias from previous epochs).

In [91]:
model = keras.models.Sequential([
    keras.layers.GRU(128, batch_input_shape=[batch_size, None, max_id], 
                     return_sequences=True, stateful=True, 
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, 
                     return_sequences=True, stateful=True, 
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, 
                                                    activation="softmax"))
])

In [92]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (32, None, 128)           64896     
                                                                 
 gru_1 (GRU)                 (32, None, 128)           99072     
                                                                 
 time_distributed (TimeDistr  (32, None, 39)           5031      
 ibuted)                                                         
                                                                 
Total params: 168,999
Trainable params: 168,999
Non-trainable params: 0
_________________________________________________________________


In [93]:
# reset states between epochs
class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="nadam",
              metrics=["accuracy"])

history = model.fit(dataset, epochs=50,
                    callbacks=[ResetStatesCallback()])

Epoch 1/50
313/313 [==============================] - 37s 108ms/step - loss: 2.5950 - accuracy: 0.2640
Epoch 2/50
313/313 [==============================] - 35s 112ms/step - loss: 2.2225 - accuracy: 0.3486
Epoch 3/50
313/313 [==============================] - 35s 112ms/step - loss: 2.2374 - accuracy: 0.3464
Epoch 4/50
313/313 [==============================] - 33s 105ms/step - loss: 2.3146 - accuracy: 0.3284
Epoch 5/50
313/313 [==============================] - 33s 106ms/step - loss: 1.9202 - accuracy: 0.4258
Epoch 6/50
313/313 [==============================] - 33s 105ms/step - loss: 1.8669 - accuracy: 0.4409
Epoch 7/50
313/313 [==============================] - 32s 102ms/step - loss: 1.8276 - accuracy: 0.4509
Epoch 8/50
313/313 [==============================] - 32s 104ms/step - loss: 1.7973 - accuracy: 0.4590
Epoch 9/50
313/313 [==============================] - 32s 101ms/step - loss: 1.7740 - accuracy: 0.4650
Epoch 10/50
313/313 [==============================] - 31s 100ms/step - l

After training this, let's now save the weights and copy to the stateless model (so we can predict on batches of any size, not just 32).

In [102]:
SAVE_DIR = os.path.join(".", "_models", "15_nlp")
os.makedirs(SAVE_DIR, exist_ok=True)
model.save_weights(os.path.join(SAVE_DIR, "shakespeare_rnn_weights.h5"))

In [103]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, 
                                                    activation="softmax"))
])

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="nadam",
              metrics=["accuracy"])

In [104]:
model.load_weights(os.path.join(SAVE_DIR, "shakespeare_rnn_weights.h5"))

### Generate Text

To generate new shakespearean text:
- Preprocess input text
    - text to seq encode, subtract 1 from indices, to one hot
- Predict next char probas
- Randomly select a char using probas
    - Add a temperature var to control how much variability you want
- Convert selected char index to char
    - Add 1, seq to text

In [118]:
def preprocess(text):
    encoded_text = np.array(tokenizer.texts_to_sequences([text]))
    return tf.one_hot(encoded_text - 1, max_id)

def next_char(model, text, temperature):
    X_new = preprocess(text)
    y_proba = model.predict(X_new, verbose=0)[0, -1:, :] # first batch, last step's output (aka last char), all probas
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

Now let's create a function that generates `n_chars` chars of shakespearean text.

In [119]:
def shakespearean_text(model, seed_text, n_chars=50, temperature=1):
    text = seed_text
    for _ in range(n_chars):
        text += next_char(model, text, temperature)
    return text

Let's try it out!

In [123]:
seed_text = "t"
print(shakespearean_text(model, seed_text, temperature=0.1))

ter:
the seem the contraction of the court of the c


In [124]:
seed_text = "t"
print(shakespearean_text(model, seed_text, temperature=1))

t, bear for the
good more one with your triar:
nor,


In [122]:
seed_text = "t"
print(shakespearean_text(model, seed_text, temperature=2))

t, ly yearss, margand?
ty-murr incewertess, kness y


Cool! Let's try using a longer seed.

In [126]:
seed_text = "Hamlet said thou are"
print(shakespearean_text(model, seed_text, n_chars=300, temperature=1))

Hamlet said thou are
since to serve go alive a timplint of such gentlemans,
the suition warwick. then!
him thou best one thy fortune one hisband.

all:
i will go should being give my cieizen's or thou
so cursed wife with a a.---

bygy:
aid you had be rather, our conession that?

sicinius:
ha, sheve i horse of a lunious


## IMDB Review Sentiment Analysis

### Get Data

In [136]:
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)

In [130]:
info.splits

{Split('train'): <SplitInfo num_examples=25000, num_shards=1>,
 Split('test'): <SplitInfo num_examples=25000, num_shards=1>,
 Split('unsupervised'): <SplitInfo num_examples=50000, num_shards=1>}

In [134]:
train_size = info.splits["train"].num_examples
train_size

25000

Let's look at what the input data looks like.

In [142]:
datasets

{'train': <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 'test': <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 'unsupervised': <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}

In [143]:
dataset = datasets["train"]

In [150]:
for x, y in dataset.take(100).shuffle(100):
    pass
x

<tf.Tensor: shape=(), dtype=string, numpy=b'In this film we have the fabulous opportunity to see what happened to Timon and Pumbaa in the film when they are not shown - which is a lot! This film even goes back to before Simba and (presumbably) just after the birth of Kiara. <br /><br />Quite true to the first film, "Lion King 1/2 (or Lion King 3 in other places)" is a funny, entertaining, exciting and surprising film (or sequel if that\'s what you want to call it). A bundle of surprises and hilarity await for you!<br /><br />While Timon and Pumbaa are watching a film at the cinema (with a remote control), Timon and Pumbaa have an argument of what point of "The Lion King" they are going to start watching, as Timon wants to go to the part when he and Pumbaa come in and Pumbaa wants to go back to the beginning. They have a very fair compromise of watching the film of their own story, which is what awaits... It starts with Timon\'s first home...<br /><br />For anyone with a good sense of h

We need to preprocess the text.

In [152]:
def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300) # truncate every review to first 300 chars
    X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ") # replace <br /> with whitespace
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ") # replace anything that is not a-z, A-Z, or ' 
                                                                     # with whitespace
    X_batch = tf.strings.split(X_batch) # create a sequence of words split by whitespace
    return X_batch.to_tensor(default_value=b"<pad>"), y_batch # pad uneven ragged tensor so that we will have a dense tensor

Now let's encode the words to ids
- Create a vocabulary w/ Counter
- Create a lookup table

Create a vocabulary in descending order most common to least common words.

In [157]:
batch_size = 32
vocabulary = Counter()
for X_batch, y_batch in dataset.batch(batch_size).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [170]:
vocabulary.most_common(5)

[(b'<pad>', 214309),
 (b'the', 61137),
 (b'a', 38564),
 (b'of', 33983),
 (b'and', 33431)]

Looks about right.

In [163]:
len(vocabulary)

53893

In [171]:
vocabulary.most_common(10005)[-5:]

[(b'Legion', 7),
 (b'Republic', 7),
 (b'Cassie', 7),
 (b'hallucinations', 7),
 (b'Clinton', 7)]

Let's just use the 10000 most common and use 1000 oov buckets for any other possible characters (probably don't even need 1000 since each sequence is only 300 words max).

In [172]:
vocab_size = 10000
truncated_vocabulary = [word for word, count in vocabulary.most_common(vocab_size)]

In [173]:
len(truncated_vocabulary)

10000

In [174]:
truncated_vocabulary[:5]

[b'<pad>', b'the', b'a', b'of', b'and']

Create a lookup table initializer.

In [176]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)

Create a lookup table.

In [177]:
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [185]:
review = tf.constant([b"This movie sucks real fucking ass man <pad>".split()])
table.lookup(review)

<tf.Tensor: shape=(1, 8), dtype=int64, numpy=array([[   22,    12,  1488,   175, 10716,  1831,   167,     0]])>

Now let's map the lookup table to each input sequence.

In [187]:
def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

### Create Datasets

Now let's wrap everything up together
- Shuffle data
- Batch the data
- Preprocess text
- Encode words
- Prefetch

In [228]:
train, valid, test = tfds.load("imdb_reviews", as_supervised=True, split=["train", "test[:50%]", "test[50%:]"])

In [229]:
batch_size = 32
train_set = train.shuffle(10000).batch(batch_size).map(preprocess).map(encode_words).prefetch(1)
valid_set = valid.batch(batch_size).map(preprocess).map(encode_words).prefetch(1)
test_set = test.batch(batch_size).map(preprocess).map(encode_words).prefetch(1)

### Model

We need to embed our word sequences for the model to learn good patterns and fast.
- Input (batch_size, seq_length)
- Embed (batch_size, seq_length, embed_dimen)
- GRU
- GRU
- Dense Binary final output only (batch_size)

In [216]:
embed_dimen = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, 
                          embed_dimen, input_shape=[None]),
    keras.layers.Conv1D(filters=embed_dimen, kernel_size=2, strides=2),
    keras.layers.BatchNormalization(),
    keras.layers.GRU(128, return_sequences=True, 
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, 
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.Dense(1, activation="sigmoid")
])

In [217]:
model.compile(loss="binary_crossentropy",
              optimizer="nadam",
              metrics=["accuracy"])

In [218]:
model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 53s 62ms/step - loss: 0.5647 - accuracy: 0.6964
Epoch 2/5
782/782 [==============================] - 50s 64ms/step - loss: 0.3691 - accuracy: 0.8373
Epoch 3/5
782/782 [==============================] - 49s 63ms/step - loss: 0.2288 - accuracy: 0.9086
Epoch 4/5
782/782 [==============================] - 49s 62ms/step - loss: 0.1298 - accuracy: 0.9524
Epoch 5/5
782/782 [==============================] - 48s 62ms/step - loss: 0.0761 - accuracy: 0.9731


In [219]:
model.evaluate(test_set)

782/782 [==============================] - 9s 10ms/step - loss: 1.0188 - accuracy: 0.7404


[1.018828272819519, 0.7404000163078308]

Seems like it overfit. Let's use masks to get rid of padding from the learning process.

### Model Masking

Using a mask to omit input values that equal 0, (id 0 is `<pad>`)

In [230]:
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs) # returns True for all non zero inputs
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_dimen)(inputs)
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation="sigmoid")(z)
model = keras.Model(inputs=[inputs], outputs=[outputs])

In [231]:
model.compile(loss="binary_crossentropy",
              optimizer="nadam",
              metrics=["accuracy"])

Let's add tensorboard callback for visualization.

In [232]:
TENSORBOARD_DIR = os.path.join(".", "_tf_logs", "15_nlp")
os.makedirs(TENSORBOARD_DIR, exist_ok=True)

In [233]:
run_index = 1
run_logdir = os.path.join(TENSORBOARD_DIR, "imdb_masked_model_run_{:05d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(log_dir=run_logdir, embeddings_freq=10)
print(run_logdir)

./_tf_logs/15_nlp/imdb_masked_model_run_00001


In [234]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5)
lr_scheduler_cb = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=1)

In [235]:
model.fit(train_set, epochs=10,
          validation_data=valid_set,
          callbacks=[tensorboard_cb, early_stopping_cb, lr_scheduler_cb])

Epoch 1/10
782/782 [==============================] - 82s 96ms/step - loss: 0.5569 - accuracy: 0.7042 - val_loss: 0.4925 - val_accuracy: 0.7590 - lr: 0.0010
Epoch 2/10
782/782 [==============================] - 69s 88ms/step - loss: 0.3775 - accuracy: 0.8358 - val_loss: 0.4883 - val_accuracy: 0.7622 - lr: 0.0010
Epoch 3/10
782/782 [==============================] - 68s 87ms/step - loss: 0.2726 - accuracy: 0.8907 - val_loss: 0.5749 - val_accuracy: 0.7567 - lr: 0.0010
Epoch 4/10
782/782 [==============================] - 68s 87ms/step - loss: 0.1348 - accuracy: 0.9521 - val_loss: 0.8755 - val_accuracy: 0.7502 - lr: 5.0000e-04
Epoch 5/10
782/782 [==============================] - 72s 92ms/step - loss: 0.0601 - accuracy: 0.9823 - val_loss: 1.0464 - val_accuracy: 0.7469 - lr: 2.5000e-04
Epoch 6/10
782/782 [==============================] - 69s 89ms/step - loss: 0.0301 - accuracy: 0.9933 - val_loss: 1.2488 - val_accuracy: 0.7455 - lr: 1.2500e-04
Epoch 7/10
782/782 [==========================

Using tensorboard, we can visualize the closest embeddings to the two words below. Let's see what words their neighbors are!

In [244]:
sample_words = tf.constant(["amazing", "awful"])
table.lookup(sample_words)

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([438, 260])>

In [250]:
# 100 closest neighbors including amazing itself
amazing_neighbor_ids = [438,674,332,327,269,284,325,1498,813,93,687,107,1573,2719,1291,577,902,870,1921,751,68,783,10126,2304,2197,2554,3101,1238,1015,3369,2474,7271,5496,2811,592,2440,2756,3093,1744,142,3530,2830,849,5639,434,6034,3925,3665,2463,2359,335,3454,1172,6188,4264,376,3762,1982,8960,8737,1196,3977,1597,276,1519,10035,2489,2169,1124,3051,4542,2772,2672,1166,579,1499,5808,1030,3350,2821,6080,2075,1040,4482,1542,3559,2931,3173,5640,4556,1618,7308,2137,3792,6371,8059,8326,9408,695,5392,3180]

# 100 closest neighbors including awful itself
awful_neighbor_ids = [260,143,1044,302,396,981,731,313,420,450,1069,1256,388,1572,788,2503,972,1669,1724,537,2893,631,966,623,1329,1619,908,3175,2523,2912,2786,460,2149,2261,1488,1129,749,391,988,1927,221,514,498,1339,3829,2841,1882,853,633,1200,1361,1052,10838,4227,1657,4736,8873,3082,2364,1037,2530,1701,1184,5948,5242,1022,1814,2232,2500,3886,2386,2621,3116,624,3512,880,1702,805,4708,1464,5311,728,1666,2736,3599,961,4834,6331,2130,5558,1065,2920,493,4003,5134,2432,356,3305,3730,5277,8640]


Now let's see what words they are!

In [262]:
amazing_neighbors = [str(truncated_vocabulary[id_]) for id_ in amazing_neighbor_ids if id_ < 10000]
awful_neighbors = [str(truncated_vocabulary[id_]) for id_ in awful_neighbor_ids if id_ < 10000]

In [263]:
for amazing_neighbor, awful_neighbor in zip(amazing_neighbors, awful_neighbors):
    print("{:20} {}".format(amazing_neighbor, awful_neighbor))

b'amazing'           b'awful'
b'fantastic'         b'worst'
b'favorite'          b'pathetic'
b'wonderful'         b'terrible'
b'excellent'         b'worse'
b'loved'             b'fails'
b'enjoyed'           b'poorly'
b'underrated'        b'boring'
b'unique'            b'horrible'
b'best'              b'waste'
b'enjoyable'         b'dumb'
b'love'              b'laughable'
b'wonderfully'       b'stupid'
b'extraordinary'     b'unfunny'
b'touching'          b'weak'
b'simple'            b'tedious'
b'awesome'           b'disappointment'
b'Great'             b'bottom'
b'feelings'          b'promising'
b'masterpiece'       b'crap'
b'great'             b'WORST'
b'superb'            b'Unfortunately'
b'refreshing'        b'mess'
b'Excellent'         b'ridiculous'
b'captured'          b'redeeming'
b'Wonderful'         b'lacks'
b'plenty'            b'badly'
b'funniest'          b'stinker'
b'stellar'           b'Worst'
b'friendship'        b'turkey'
b'limitations'       b'idiotic'
b'advanced'       

Wow, embeddings really do work!!!

### Pretrained Embeddings

Let's use Google's pretrained model for english sentence embedding.

This model does not require preprocessing, so let's create new datasets.

In [316]:
train, valid, test = tfds.load("imdb_reviews", as_supervised=True, 
                               split=["train+test[:50%]", "test[50%:75%]", "test[75%:]"])

batch_size = 32
train_set = train.shuffle(10000).batch(batch_size).prefetch(1)
valid_set = valid.batch(batch_size).prefetch(1)
test_set = test.batch(batch_size).prefetch(1)

Now let's create our model using the tfhub model.

In [317]:
os.environ["TFHUB_CACHE_DIR"] = os.path.join(".", "my_tfhub_cache")

In [318]:
model = keras.models.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1", output_shape=[50],
                   input_shape=[], dtype=tf.string),
    keras.layers.Dense(150, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(1, activation="sigmoid"),
])

In [320]:
model.compile(loss="binary_crossentropy",
              optimizer="nadam",
              metrics=["accuracy"])

In [321]:
model.fit(train_set, epochs=5,
          validation_data=valid_set)

Epoch 1/5
1172/1172 [==============================] - 6s 4ms/step - loss: 0.5538 - accuracy: 0.7198 - val_loss: 0.5285 - val_accuracy: 0.7402
Epoch 2/5
1172/1172 [==============================] - 4s 3ms/step - loss: 0.5259 - accuracy: 0.7393 - val_loss: 0.5220 - val_accuracy: 0.7470
Epoch 3/5
1172/1172 [==============================] - 4s 3ms/step - loss: 0.5201 - accuracy: 0.7456 - val_loss: 0.5104 - val_accuracy: 0.7491
Epoch 4/5
1172/1172 [==============================] - 4s 3ms/step - loss: 0.5165 - accuracy: 0.7462 - val_loss: 0.5126 - val_accuracy: 0.7507
Epoch 5/5
1172/1172 [==============================] - 4s 3ms/step - loss: 0.5140 - accuracy: 0.7477 - val_loss: 0.5543 - val_accuracy: 0.7232


In [323]:
model.layers[0].trainable = True

In [324]:
run_index = 6
run_logdir = os.path.join(TENSORBOARD_DIR, "imdb_tfhub_model_run_{:05d}".format(run_index))
tensorboard_cb = keras.callbacks.TensorBoard(log_dir=run_logdir, embeddings_freq=2)
print(run_logdir)

./_tf_logs/15_nlp/imdb_tfhub_model_run_00006


In [325]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10)
lr_scheduler_cb = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=2)

In [326]:
model.fit(train_set, epochs=100,
          validation_data=valid_set,
          callbacks=[tensorboard_cb, early_stopping_cb, lr_scheduler_cb])

Epoch 1/100
1172/1172 [==============================] - 5s 4ms/step - loss: 0.5129 - accuracy: 0.7501 - val_loss: 0.5192 - val_accuracy: 0.7498 - lr: 0.0010
Epoch 2/100
1172/1172 [==============================] - 4s 3ms/step - loss: 0.5115 - accuracy: 0.7498 - val_loss: 0.5135 - val_accuracy: 0.7514 - lr: 0.0010
Epoch 3/100
1172/1172 [==============================] - 5s 4ms/step - loss: 0.5101 - accuracy: 0.7516 - val_loss: 0.5100 - val_accuracy: 0.7504 - lr: 0.0010
Epoch 4/100
1172/1172 [==============================] - 4s 3ms/step - loss: 0.5080 - accuracy: 0.7519 - val_loss: 0.5195 - val_accuracy: 0.7443 - lr: 0.0010
Epoch 5/100
1172/1172 [==============================] - 5s 4ms/step - loss: 0.5074 - accuracy: 0.7530 - val_loss: 0.5140 - val_accuracy: 0.7477 - lr: 0.0010
Epoch 6/100
1172/1172 [==============================] - 4s 3ms/step - loss: 0.5025 - accuracy: 0.7535 - val_loss: 0.5072 - val_accuracy: 0.7525 - lr: 5.0000e-04
Epoch 7/100
1172/1172 [=========================

In [331]:
model.predict(["I really don't like this movie. It's horrible, terrible, disgusting, awful, bad."])

1/1 [==============================] - 0s 39ms/step


array([[0.08318325]], dtype=float32)

In [328]:
model.predict(["This movie is great! I love her so much! Amazing!"])

1/1 [==============================] - 0s 42ms/step


array([[0.9896085]], dtype=float32)

# Exercises

## Embedded Reber Grammars

We need to first generate our dataset.

### Get Data

Using random selection with uniform probabilities, let's try to create a function that generates an embedded reber grammar string.

![reber_grammar](_images/reber_grammar.gif)

Each state represents an edge or edges of a vertice in a directed graph. Except the edges have values and the vertices don't.

In [508]:
default_reber_grammar = [
    [("B", 1)], # starting edge: (B to vertex 1)
    [("T", 2), ("P", 3)], # vertex 1 has two outgoing edges: (T to vertex 2) and (P to vertex 3)
    [("S", 2), ("X", 4)], # vertex 2 has two outgoing edges: (S back to vertex 2) and (X to vertex 4)
    [("T", 3), ("V", 5)], # vertex 3 has two outgoing edges: (T back to vertex 3) and (V to vertex 5)
    [("X", 3), ("S", 6)], # vertex 4 has two outgoing edges: (X to vertex 3) and (S to vertex 6)
    [("P", 4), ("V", 6)], # vertex 5 has two outgoing edges: (P to vertex 4) and (V to vertex 6)
    [("E", None)], # vertex 6 has one outgoing edge: (E to end)
]

def generate_reber_string(grammar=default_reber_grammar):
    vertex = 0 # starting from first edge (no vertex)
    output = ""
    while vertex != None: # iterate until vertex = None
        n_edges = len(grammar[vertex]) # how many edges does our current vertex have?
        edge = np.random.randint(n_edges) # if n_edges = 1, then edge will be 0 only
        reber_value, vertex = grammar[vertex][edge] # get edge's value and update vertex to vertex it's pointing to
        output += reber_value
    return output

In [509]:
for _ in range(5):
    print(generate_reber_string())

BTXSE
BPTVPSE
BTXXVPXTTVPXVVE
BPTVPXVVE
BTSXSE


Now let's use the embedded graph.

In [510]:
embedded_reber_grammar = [
    [("B", 1)], # starting edge: (B to vertex 1)
    [("T", 2), ("P", 3)], # vertex 1 has two outgoing edges: (T to vertex 2) and (P to vertex 3)
    [(default_reber_grammar, 4)], # vertex 2 has one outgoing edge: (embedded reber to vertex 4)
    [(default_reber_grammar, 5)], # vertex 3 has one outgoing edge: (embedded reber to vertex 5)
    [("T", 6)], # vertex 4 has one outgoing edge: (T to vertex 6)
    [("P", 6)], # vertex 5 has one outgoing edge: (P to vertex 6)
    [("E", None)], # vertex 6 has one outgoing edge: (E to end)
]

We need to update our function so that it can recursively create reber strings if reber grammar is embedded.

In [511]:
def generate_reber_string(grammar=embedded_reber_grammar):
    vertex = 0 # starting from first edge (no vertex)
    output = ""
    while vertex != None: # iterate until vertex = None
        n_edges = len(grammar[vertex]) # how many edges does our current vertex have?
        edge = np.random.randint(n_edges) # if n_edges = 1, then edge will be 0 only
        reber_value, vertex = grammar[vertex][edge] # get edge's value and update vertex to vertex it's pointing to
        # if reber_value is a list - then it's an embedded graph
        if isinstance(reber_value, list):
            reber_value = generate_reber_string(reber_value) # we want to update value to the resulting reber string
        output += reber_value
    return output

In [512]:
for _ in range(5):
    print(generate_reber_string())

BTBPTVVETE
BPBTSXSEPE
BTBPTTTVPSETE
BPBPTVPSEPE
BTBPTVVETE


Now we need to be able to generate strings that fail to follow the embedded reber grammar. There are many ways to do this, but what is the best way? The way that can cover the widest range? Generalize best?

A good way is to randomly change one char from a good string
- Will present enough difficulty
- Will cover a wide ranges of possible strings

In [513]:
POSSIBLE_CHARS = "BEPSTVX"

def generate_corrupted_string(grammar=embedded_reber_grammar, chars=POSSIBLE_CHARS):
    good_string = generate_string(grammar)
    index = np.random.randint(len(good_string))
    good_char = good_string[index]
    bad_char = np.random.choice(sorted(set(chars) - set(good_char))) # randomly select a char that is different
    return good_string[:index] + bad_char + good_string[index + 1:]

In [514]:
for _ in range(5):
    print(generate_corrupted_string())

BTBPTVPSXTE
BTBPTVPSVVETE
BPBTSTXTVPSEPE
BXBPVPSEPE
BPBPVPSEPV


### Create Datasets

We want our train, valid, and test sets to have an equal number of positive and negative samples, so we must create a pos. list of tuples (string, 1) and a neg. list of tuples (string, 0) for each set.

We need to encode the strings to their indices so that we can embed the chars in our model.

In [515]:
def string_to_ids(s, chars=POSSIBLE_CHARS):
    return [chars.index(c) for c in s]

In [516]:
def reber_dataset(size):
    pos = [string_to_ids(generate_reber_string())
           for _ in range(size // 2)]
    neg = [string_to_ids(generate_corrupted_string())
           for _ in range(size - size // 2)]
    X = tf.ragged.constant(pos + neg, ragged_rank=1)
    y = tf.constant(([1.] * len(pos)) + ([0.] * len(neg)))
    return X, y

In [521]:
reset_backend()

train_set = tf.data.Dataset.from_tensor_slices(reber_dataset(20000)).shuffle(20000).batch(32).prefetch(1)
valid_set = tf.data.Dataset.from_tensor_slices(reber_dataset(5000)).batch(32).prefetch(1)

Now let's create our model.

### Model

- Embed input
- GRU
- Dense with 1 unit and activation is sigmoid

In [527]:
reset_backend()

embed_dim = 5

model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=[None], dtype=tf.int32, ragged=True),
    keras.layers.Embedding(input_dim=len(POSSIBLE_CHARS), output_dim=embed_dim),
    keras.layers.GRU(30),
    keras.layers.Dense(1, activation="sigmoid")
])

In [528]:
optimizer = keras.optimizers.SGD(learning_rate=0.02, momentum = 0.95, nesterov=True)
model.compile(loss="binary_crossentropy", 
              optimizer=optimizer, 
              metrics=["accuracy"])

history = model.fit(train_set, epochs=20, 
                    validation_data=valid_set)

Epoch 1/20
625/625 [==============================] - 8s 9ms/step - loss: 0.6794 - accuracy: 0.5433 - val_loss: 0.6532 - val_accuracy: 0.6138
Epoch 2/20
625/625 [==============================] - 5s 8ms/step - loss: 0.6416 - accuracy: 0.5964 - val_loss: 0.6193 - val_accuracy: 0.6264
Epoch 3/20
625/625 [==============================] - 5s 8ms/step - loss: 0.5772 - accuracy: 0.6683 - val_loss: 0.5322 - val_accuracy: 0.7602
Epoch 4/20
625/625 [==============================] - 5s 8ms/step - loss: 0.3928 - accuracy: 0.8228 - val_loss: 0.2474 - val_accuracy: 0.9034
Epoch 5/20
625/625 [==============================] - 5s 8ms/step - loss: 0.1133 - accuracy: 0.9679 - val_loss: 0.0781 - val_accuracy: 0.9824
Epoch 6/20
625/625 [==============================] - 5s 8ms/step - loss: 0.0841 - accuracy: 0.9797 - val_loss: 0.0562 - val_accuracy: 0.9896
Epoch 7/20
625/625 [==============================] - 5s 8ms/step - loss: 0.0240 - accuracy: 0.9948 - val_loss: 0.0016 - val_accuracy: 1.0000
Epoch 

In [529]:
test_strings = ["BPBTSSSSSSSXXTTVPXVPXTTTTTVVETE",
                "BPBTSSSSSSSXXTTVPXVPXTTTTTVVEPE"]
X_test = tf.ragged.constant([string_to_ids(s) for s in test_strings], ragged_rank=1)

y_proba = model.predict(X_test)
print()
print("Estimated probability that these are Reber strings:")
for index, string in enumerate(test_strings):
    print("{}: {:.2f}%".format(string, 100 * y_proba[index][0]))

1/1 [==============================] - 0s 345ms/step

Estimated probability that these are Reber strings:
BPBTSSSSSSSXXTTVPXVPXTTTTTVVETE: 0.01%
BPBTSSSSSSSXXTTVPXVPXTTTTTVVEPE: 99.98%


### Mistakes

- I did not double check my data generation function and my embedded grammar was wrong!
- I used a way too complicated model - next time use a simple model and gradually increase complexity if the simple model doesn't work!

## Convert Date String

We first need to generate date string data.

### Get Data

- Generate Random month, day, year
- Return same date in input and output formats
- Encode input and output strings

In [532]:
from datetime import date

# year, month, day
date(1000, 1, 1).toordinal()

364878

In [534]:
date(9999, 12, 31).toordinal()

3652059

In [535]:
date.fromordinal(3652059 - 100000)

datetime.date(9726, 3, 17)

To generate our data
- Set max and min dates
- Numpy random to get random date btwn max and min
- Format resulting date to string
    - Input: `"April 22, 2019"`
    - Target: `"2019-04-22"`

In [562]:
MONTHS = ["January", "February", "March", "April", "May", "June",
          "July", "August", "September", "October", "November", "December"]

def random_dates(n_dates):
    min_date = date(1000, 1, 1).toordinal()
    max_date = date(9999, 12, 31).toordinal()
    
    # select random date btwn min and max
    ordinals = np.random.randint(max_date-min_date, size=n_dates) + min_date
    dates = [date.fromordinal(ordinal) for ordinal in ordinals]
    
    # date to string formats
    X = [MONTHS[dt.month - 1] + " " + dt.strftime("%d, %Y") for dt in dates]
    y = [dt.isoformat() for dt in dates]
    
    return X, y

In [670]:
n_dates = 3
x_example, y_example = random_dates(n_dates)
print("{:25s}{:25s}".format("Input", "Target"))
print("-" * 50)
for idx in range(n_dates):
    print("{:25s}{:25s}".format(x_example[idx], y_example[idx]))

Input                    Target                   
--------------------------------------------------
March 26, 4682           4682-03-26               
March 03, 8128           8128-03-03               
December 23, 4998        4998-12-23               


Now we need to encode our data.

In [575]:
INPUT_CHARS = "".join(sorted(set("".join(MONTHS) + "0123456789, ")))
INPUT_CHARS

' ,0123456789ADFJMNOSabceghilmnoprstuvy'

In [576]:
OUTPUT_CHARS = "0123456789-"

In [578]:
def date_str_to_ids(date_str, chars):
    return [chars.index(c) for c in date_str]

In [587]:
sample_input, sample_output = random_dates(1)
print(sample_input)
print(sample_output)

['June 20, 4110']
['4110-06-20']


In [588]:
date_str_to_ids(sample_input[0], INPUT_CHARS)

[15, 35, 29, 23, 0, 4, 2, 1, 0, 6, 3, 3, 2]

In [589]:
date_str_to_ids(sample_output[0], OUTPUT_CHARS)

[4, 1, 1, 0, 10, 0, 6, 10, 2, 0]

### Create Datasets

- Generate X and y len = `size` for train and valid
- Shuffle train
- Batch and prefetch

In [709]:
def encode_date_strs(date_strs, chars):
    ids = [date_str_to_ids(input_str, chars) for input_str in date_strs]
    ids = tf.ragged.constant(ids, ragged_rank=1)
    return (ids + 1).to_tensor() # 0 for padding

max_input_length = 18

def prepare_date_strs_padded(date_strs):
    X = encode_date_strs(date_strs, INPUT_CHARS)
    if X.shape[1] < max_input_length:
        X = tf.pad(X, [[0, 0], [0, max_input_length - X.shape[1]]])
    return X

def date_str_dataset(size):
    X, Y = random_dates(size) # X (size, dtype=str), Y (size, dtype=str)
    
    return prepare_date_strs_padded(X), encode_date_strs(Y, OUTPUT_CHARS)

In [699]:
train_set = tf.data.Dataset.from_tensor_slices(date_str_dataset(10000)).shuffle(10000).batch(32).prefetch(1)
valid_set = tf.data.Dataset.from_tensor_slices(date_str_dataset(2000)).batch(32).prefetch(1)
test_set = tf.data.Dataset.from_tensor_slices(date_str_dataset(2000)).batch(32).prefetch(1)

In [710]:
for batch in train_set.take(1):
    pass

In [711]:
batch[0]

<tf.Tensor: shape=(32, 18), dtype=int32, numpy=
array([[16, 36, 28, 38,  1,  3,  8,  2,  1,  5,  3, 10,  9,  0,  0,  0,
         0,  0],
       [13, 32, 33, 27, 28,  1,  5, 10,  2,  1,  7, 12, 11,  5,  0,  0,
         0,  0],
       [17, 21, 38,  1,  5, 12,  2,  1, 10,  8, 10,  6,  0,  0,  0,  0,
         0,  0],
       [13, 32, 33, 27, 28,  1,  3,  4,  2,  1,  6,  5,  9, 10,  0,  0,
         0,  0],
       [13, 32, 33, 27, 28,  1,  5,  4,  2,  1,  9, 10, 12, 11,  0,  0,
         0,  0],
       [18, 31, 37, 24, 29, 22, 24, 33,  1,  3,  7,  2,  1, 11,  4,  5,
         7,  0],
       [17, 21, 33, 23, 26,  1,  3, 12,  2,  1,  5,  8,  7, 12,  0,  0,
         0,  0],
       [15, 24, 22, 33, 36, 21, 33, 38,  1,  4,  8,  2,  1, 11,  3,  4,
         3,  0],
       [15, 24, 22, 33, 36, 21, 33, 38,  1,  3, 12,  2,  1, 11,  8, 11,
         8,  0],
       [17, 21, 38,  1,  3,  7,  2,  1,  7,  8,  9,  4,  0,  0,  0,  0,
         0,  0],
       [17, 21, 38,  1,  3, 12,  2,  1,  8,  3,  7, 11,  0,  0

In [712]:
batch[1]

<tf.Tensor: shape=(32, 10), dtype=int32, numpy=
array([[ 3,  1,  8,  7, 11,  1,  8, 11,  1,  6],
       [ 5, 10,  9,  3, 11,  1,  5, 11,  3,  8],
       [ 8,  6,  8,  4, 11,  1,  6, 11,  3, 10],
       [ 4,  3,  7,  8, 11,  1,  5, 11,  1,  2],
       [ 7,  8, 10,  9, 11,  1,  5, 11,  3,  2],
       [ 9,  2,  3,  5, 11,  2,  2, 11,  1,  5],
       [ 3,  6,  5, 10, 11,  1,  4, 11,  1, 10],
       [ 9,  1,  2,  1, 11,  1,  3, 11,  2,  6],
       [ 9,  6,  9,  6, 11,  1,  3, 11,  1, 10],
       [ 5,  6,  7,  2, 11,  1,  6, 11,  1,  5],
       [ 6,  1,  5,  9, 11,  1,  6, 11,  1, 10],
       [10,  6,  3,  8, 11,  1,  3, 11,  3,  7],
       [ 9,  2, 10,  5, 11,  1,  2, 11,  3,  6],
       [ 4,  3, 10,  7, 11,  1,  8, 11,  2,  8],
       [ 7,  1,  4,  4, 11,  1, 10, 11,  1,  2],
       [ 7,  3,  6,  5, 11,  1,  4, 11,  1,  8],
       [ 2,  2,  9,  6, 11,  1,  7, 11,  1,  4],
       [ 6,  6,  1,  4, 11,  1,  3, 11,  2,  4],
       [ 6,  7,  2,  1, 11,  1,  5, 11,  1,  2],
       [ 3,  1,  5,  

### Simple Model

Let's first train a simple model that uses the most basic seq to seq architecture.
- Encoder
    - Embeds inputs
    - GRU layer with one output
- Repeat encoder's output to form seq of same length as target
- Decoder
    - GRU that outputs seq
    - Dense softmax for each output of seq

In [602]:
len(INPUT_CHARS)

38

In [619]:
len(OUTPUT_CHARS)

11

In [620]:
batch[1].shape

TensorShape([32, 10])

In [621]:
embed_dim = 32
input_dim = len(INPUT_CHARS) + 1         # padding is index 0
output_dim = len(OUTPUT_CHARS) + 1       # padding is index 0

encoder = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=[None]),
    keras.layers.Embedding(input_dim=input_dim, output_dim=embed_dim),
    keras.layers.GRU(128) # outputs last output vector of size 128
])

decoder = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(output_dim, activation="softmax")),
])

model = keras.models.Sequential([
    encoder,
    keras.layers.RepeatVector(10), # must be equal to target string length
    decoder
])

In [633]:
optimizer = keras.optimizers.Nadam(learning_rate=5e-5)
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer=optimizer,
              metrics=["accuracy"])

In [634]:
history = model.fit(train_set, epochs=1,
                    validation_data=valid_set)

625/625 [==============================] - 15s 18ms/step - loss: 2.7435e-05 - accuracy: 1.0000 - val_loss: 1.9824e-05 - val_accuracy: 1.0000


In [635]:
model.evaluate(test_set)

125/125 [==============================] - 1s 5ms/step - loss: 2.9934e-05 - accuracy: 1.0000


[2.9933540645288303e-05, 1.0]

heheheheh

In [682]:
probas = model.predict(encode_date_strs(["April 04, 2022"], INPUT_CHARS))

1/1 [==============================] - 1s 615ms/step


In [683]:
probas.shape

(1, 10, 12)

One element in batch, 10 chars, 12 possible probas for each char

In [684]:
pred = np.argmax(probas, axis=2)
pred.shape

(1, 10)

In [685]:
pred

array([[ 3,  1,  3,  3, 11,  1, 10, 11,  1,  5]])

One element in batch, 10 predicted chars according to max proba

In [686]:
def ids_to_date_strs(ids, chars=OUTPUT_CHARS):
    return ["".join([("?" + chars)[index] for index in sequence])
            for sequence in ids]

In [690]:
ids_to_date_strs(pred)

['2022-09-04']

Oof it's wrong

In [689]:
X_new = encode_date_strs(["September 17, 2009", "July 14, 1789"], INPUT_CHARS)

In [691]:
ids = np.argmax(model.predict(X_new), axis=-1)
for date_str in ids_to_date_strs(ids):
    print(date_str)

1/1 [==============================] - 0s 20ms/step
2009-09-17
1789-07-14


In [695]:
max_input_length = 18

def prepare_date_strs_padded(date_strs):
    X = encode_date_strs(date_strs, INPUT_CHARS)
    if X.shape[1] < max_input_length:
        X = tf.pad(X, [[0, 0], [0, max_input_length - X.shape[1]]])
    return X

def convert_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    #ids = model.predict_classes(X)
    ids = np.argmax(model.predict(X), axis=-1)
    return ids_to_date_strs(ids)

In [696]:
convert_date_strs(["May 02, 2020", "July 14, 1789"])

1/1 [==============================] - 0s 20ms/step


['2020-05-02', '1789-07-14']

### Encoder-Decoder w/ Teacher Forcing

Our previous model just repeated the encoder's output to form the decoder's input sequence, this is not optimal - let's use the previous target as the input instead (and during inference use the previous time step's prediction).

Let's first use tensors, and we can adapt into a mappable function for dataset objects later.

In [773]:
X_train, Y_train = date_str_dataset(10000)
X_valid, Y_valid = date_str_dataset(2000)
X_test, Y_test = date_str_dataset(2000)

In [794]:
sos_id = len(OUTPUT_CHARS) + 1

def shifted_output_sequences(Y):
    sos_tokens = tf.fill(dims=(len(Y), 1), value=sos_id)
    return tf.concat([sos_tokens, Y[:, :-1]], axis=1)

In [795]:
X_train_decoder = shifted_output_sequences(Y_train)
X_valid_decoder = shifted_output_sequences(Y_valid)
X_test_decoder = shifted_output_sequences(Y_test)

We do not need last char for the decoder input - we still have the last char in the target data.

In [802]:
def date_str_encoder_decoder_dataset(size):
    X, Y = random_dates(size) # X (size, dtype=str), Y (size, dtype=str)
    X_ids_padded = prepare_date_strs_padded(X)
    Y_ids = encode_date_strs(Y, OUTPUT_CHARS)
    Y_shifted = shifted_output_sequences(Y_ids)
    
    return (X_ids_padded, Y_shifted), Y_ids

In [806]:
train_set = tf.data.Dataset.from_tensor_slices(
    date_str_encoder_decoder_dataset(10000)).shuffle(10000).batch(32).prefetch(1)

valid_set = tf.data.Dataset.from_tensor_slices(date_str_encoder_decoder_dataset(2000)).batch(32).prefetch(1)
test_set = tf.data.Dataset.from_tensor_slices(date_str_encoder_decoder_dataset(2000)).batch(32).prefetch(1)

Now let's bulid our encoder-decoder model that takes the previous step's target as input for decoder.

In [809]:
encoder_embed_dim = 32
decoder_embed_dim = 32
n_gru_units = 128

K.clear_session()
np.random.seed(SEED)
tf.random.set_seed(SEED)


# encoder input: takes date str id seq
encoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)
# encoder embed: embed seq id to meaningful vector
encoder_embedding = keras.layers.Embedding(
    input_dim=len(INPUT_CHARS) + 1, 
    output_dim=encoder_embed_dim)(encoder_input)
# encoder
encoder = keras.layers.GRU(n_gru_units, return_state=True)
encoder_output, encoder_state_h = encoder(encoder_embedding)
# encoder state (for LSTM this would be state_h and state_c)
encoder_state = encoder_state_h


# decoder input: takes shifted target id seq
decoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)
# decoder embed: embed seq id to meaningful vector
decoder_embedding = keras.layers.Embedding(
    input_dim=len(OUTPUT_CHARS) + 2,
    output_dim=decoder_embed_dim)(decoder_input)
# decoder gru output
decoder_gru_output = keras.layers.GRU(n_gru_units, return_sequences=True)(
    decoder_embedding, initial_state=encoder_state)
# final decoder proba output
decoder_output = keras.layers.TimeDistributed(
    keras.layers.Dense(len(OUTPUT_CHARS) + 1, activation="softmax"))(decoder_gru_output)


# model
model = keras.models.Model(inputs=[encoder_input, decoder_input],
                           outputs=[decoder_output])

Basically: encoder (input date str id seq) -> pass state to decoder -> decoder (shifted target id seq) -> output target id seq

In [810]:
optimizer = keras.optimizers.Nadam(learning_rate=5e-3)
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer=optimizer,
              metrics=["accuracy"])

In [811]:
model.fit(train_set, epochs=20,
          validation_data=valid_set)

Epoch 1/20
313/313 [==============================] - 9s 18ms/step - loss: 1.1362 - accuracy: 0.5633 - val_loss: 0.5468 - val_accuracy: 0.7740
Epoch 2/20
313/313 [==============================] - 5s 16ms/step - loss: 0.4683 - accuracy: 0.7982 - val_loss: 0.3949 - val_accuracy: 0.8328
Epoch 3/20
313/313 [==============================] - 5s 16ms/step - loss: 0.4119 - accuracy: 0.8269 - val_loss: 0.3579 - val_accuracy: 0.8499
Epoch 4/20
313/313 [==============================] - 5s 15ms/step - loss: 0.3230 - accuracy: 0.8647 - val_loss: 0.3023 - val_accuracy: 0.8723
Epoch 5/20
313/313 [==============================] - 5s 16ms/step - loss: 0.2925 - accuracy: 0.8817 - val_loss: 0.2500 - val_accuracy: 0.9034
Epoch 6/20
313/313 [==============================] - 5s 15ms/step - loss: 0.2256 - accuracy: 0.9154 - val_loss: 0.1845 - val_accuracy: 0.9274
Epoch 7/20
313/313 [==============================] - 5s 17ms/step - loss: 0.1459 - accuracy: 0.9426 - val_loss: 0.1106 - val_accuracy: 0.9568

In [812]:
model.evaluate(test_set)

63/63 [==============================] - 0s 5ms/step - loss: 1.1494e-04 - accuracy: 1.0000


[0.00011493735655676574, 1.0]

Great! Let's test it in action!

In [835]:
sos_id = len(OUTPUT_CHARS) + 1
max_output_length = 10

def predict_date_strs(date_strs):
    """
    Take a list of date strings like ["July 14, 1789", "May 01, 2020"] and 
    output their converted formats using our encoder-decoder model.
    
    Ex.
    In:  predict_date_strs(["July 14, 1789", "May 01, 2020"])
    Out: ['1789-07-14', '2020-05-01']
    """
    
    # get proper seq inputs for model
    X = prepare_date_strs_padded(date_strs) # encoder input
    Y_pred = tf.fill(dims=(len(date_strs), 1), value=sos_id) # decoder input non padded (one char at a time, starting with sos)
    
    # iterate till full prediction
    for index in range(max_output_length):
        # pad Y_pred with 0s till seq length
        pad_size = max_output_length - Y_pred.shape[-1]
        X_decoder = tf.pad(Y_pred, [[0, 0], [0, pad_size]])
        Y_probas_next = model.predict((X, X_decoder), verbose=0)[:, index:index+1] # predicted char
        Y_pred_next = tf.argmax(Y_probas_next, axis=-1, output_type=tf.int32) # get max out of 12 possibile chars
        Y_pred = tf.concat([Y_pred, Y_pred_next], axis=-1)
    return ids_to_date_strs(Y_pred[:, 1:])

In [837]:
predict_date_strs(["July 14, 1789", "May 01, 2020"])

['1789-07-14', '2020-05-01']

Nice! Take it slow but fully understand things! That's the way to learn!

### Using tfa.seq2seq

The third implementation of encoder-decoder, let's use tensorflow-addon's seq2seq API.

With the normal training sampler, there's really no differene btwn seq2seq and our above implementation using just Functional API. But if we use GreedyEmbeddingSampler, it will automatically use the argmax of the outputted probas as the input for the next time step. This will mean much easier inferencing.

In [858]:
K.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

encoder_embedding_size = 32
decoder_embedding_size = 32
units = 128


# encoder
encoder_inputs = keras.layers.Input(shape=[None], dtype=tf.int32)
encoder_embeddings = keras.layers.Embedding(
    len(INPUT_CHARS) + 1, encoder_embedding_size)(encoder_inputs)
encoder = keras.layers.GRU(units, return_state=True)
encoder_outputs, state_h = encoder(encoder_embeddings)
encoder_state = [state_h]


# greedy sampler
decoder_embedding_layer = keras.layers.Embedding(
    len(OUTPUT_CHARS) + 2, decoder_embedding_size)
inference_sampler = tfa.seq2seq.sampler.GreedyEmbeddingSampler(
    embedding_fn=decoder_embedding_layer)


# decoder
decoder_cell = keras.layers.GRUCell(units)
output_layer = keras.layers.Dense(len(OUTPUT_CHARS) + 1)

inference_decoder = tfa.seq2seq.basic_decoder.BasicDecoder(
    decoder_cell, inference_sampler, output_layer=output_layer,
    maximum_iterations=max_output_length) # set max iter to prevent infinite loop if decoder never outputs end token

batch_size = tf.shape(encoder_inputs)[:1]
start_tokens = tf.fill(dims=batch_size, value=sos_id) # need start tokens for decoder

final_outputs, final_state, final_sequence_lengths = inference_decoder(
    start_tokens,
    initial_state=encoder_state,
    start_tokens=start_tokens,
    end_token=0)
Y_proba = keras.layers.Activation("softmax")(final_outputs.rnn_output)


# model
model = keras.models.Model(inputs=[encoder_inputs], # only need encoder inputs now
                                     outputs=[Y_proba]) # automatically outputs the id instead of probas

In [859]:
optimizer = keras.optimizers.Nadam(learning_rate=5e-3)
model.compile(loss="sparse_categorical_crossentropy", 
                        optimizer=optimizer,
                        metrics=["accuracy"])

Since we don't need the shifted decoder inputs, let's use the data from the first model.

In [860]:
train_set = tf.data.Dataset.from_tensor_slices(date_str_dataset(10000)).shuffle(10000).batch(32).prefetch(1)
valid_set = tf.data.Dataset.from_tensor_slices(date_str_dataset(2000)).batch(32).prefetch(1)
test_set = tf.data.Dataset.from_tensor_slices(date_str_dataset(2000)).batch(32).prefetch(1)

In [861]:
model.fit(train_set, epochs=20,
                    validation_data=valid_set)

Epoch 1/20
313/313 [==============================] - 9s 18ms/step - loss: 1.3558 - accuracy: 0.5054 - val_loss: 1.2891 - val_accuracy: 0.5839
Epoch 2/20
313/313 [==============================] - 5s 16ms/step - loss: 0.5512 - accuracy: 0.7786 - val_loss: 0.4532 - val_accuracy: 0.8125
Epoch 3/20
313/313 [==============================] - 5s 16ms/step - loss: 0.3874 - accuracy: 0.8358 - val_loss: 0.3636 - val_accuracy: 0.8475
Epoch 4/20
313/313 [==============================] - 5s 16ms/step - loss: 0.3634 - accuracy: 0.8441 - val_loss: 0.3608 - val_accuracy: 0.8482
Epoch 5/20
313/313 [==============================] - 5s 16ms/step - loss: 0.3155 - accuracy: 0.8681 - val_loss: 0.2463 - val_accuracy: 0.8979
Epoch 6/20
313/313 [==============================] - 5s 17ms/step - loss: 0.1750 - accuracy: 0.9213 - val_loss: 0.1099 - val_accuracy: 0.9490
Epoch 7/20
313/313 [==============================] - 5s 17ms/step - loss: 0.0736 - accuracy: 0.9720 - val_loss: 0.0233 - val_accuracy: 0.9936

Now that we've trained the model, we can make a model reusing the trained encoder and decoder that only outputs the ids instead of probas. And, now we can directly pass the preprocessed date strings instead of having to create a shifted decoder input.

In [863]:
inference_model = keras.models.Model(inputs=[encoder_inputs],
                                     outputs=[final_outputs.sample_id])

In [865]:
ids_to_date_strs(inference_model.predict(prepare_date_strs_padded(["August 02, 2022"])))

1/1 [==============================] - 0s 19ms/step


['2022-08-02']

In [869]:
def fast_predict_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    Y_pred = inference_model.predict(X, verbose=0)
    return ids_to_date_strs(Y_pred)

In [870]:
%timeit fast_predict_date_strs(["July 14, 1789", "May 01, 2020"])

39 ms ± 3.38 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


There are many more versions:
- Train with encodre and decoder datasets (encoder is just input, decoder is target shifted right by one using sos)
    - Use scheduled sampler, can change proba of using predicted output instead of decoder input
- Inference using Sample Sampler
    - Like greedy, but has a temperature arg that allows for random choosing of output instead of argmax
- Using keras subclassing API and attention wrapper that takes in all encoder_outputs instead of just last encoder state

## Using Pretrained Model to Generate Shakespearean Text

From huggingface's transformers library.

In [871]:
from transformers import TFOpenAIGPTLMHeadModel

model = TFOpenAIGPTLMHeadModel.from_pretrained("openai-gpt")

Downloading:   0%|          | 0.00/656 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFOpenAIGPTLMHeadModel.

All the layers of TFOpenAIGPTLMHeadModel were initialized from the model checkpoint at openai-gpt.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFOpenAIGPTLMHeadModel for predictions without further training.


Now we need a tokenizer.

In [873]:
from transformers import OpenAIGPTTokenizer

tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")

Downloading:   0%|          | 0.00/797k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/448k [00:00<?, ?B/s]

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


Let's use the tokenizer.

In [874]:
prompt_text = "This royal throne of kings, this sceptred isle"
encoded_prompt = tokenizer.encode(prompt_text,
                                  add_special_tokens=False,
                                  return_tensors="tf")
encoded_prompt

<tf.Tensor: shape=(1, 10), dtype=int32, numpy=
array([[  616,  5751,  6404,   498,  9606,   240,   616, 26271,  7428,
        16187]], dtype=int32)>

Using the prompt text and model, let's generate text!

In [875]:
num_sequences = 5
length = 40

generated_sequences = model.generate(
    input_ids=encoded_prompt,
    do_sample=True,
    max_length=length + len(encoded_prompt[0]),
    temperature=1.0,
    top_k=0,
    top_p=0.9,
    repetition_penalty=1.0,
    num_return_sequences=num_sequences,
)

generated_sequences

<tf.Tensor: shape=(5, 50), dtype=int32, numpy=
array([[  616,  5751,  6404,   498,  9606,   240,   616, 26271,  7428,
        16187,   240,   246,  6404,   498,  1109,   481,   618,   488,
          481,  3016,   488,   240,  1855,   240,   246,  4206,   522,
          246,  4206,   498,   481,  4187,   240,   566,   498,  1076,
         2609,  1643,   260,  1734,  8777,   617,   984,   481,  2313,
          498,   481, 39145, 30675,   762],
       [  616,  5751,  6404,   498,  9606,   240,   616, 26271,  7428,
        16187,   980,   694, 28981,   500,   481,  6073,   498,   704,
        23376,   239, 40477,   597,   921,   704,  5176,   488,  1357,
          481,  1807,   239,  1370,   562,  6384,   617,   481,   566,
          512,  4978,   557,   618,   239, 40477,   600,  5298,   557,
          600,  1172,   481,  1002,   239],
       [  616,  5751,  6404,   498,  9606,   240,   616, 26271,  7428,
        16187,   239,   580,  2227,   485,   921,   531,  3367,   500,
          524

In [876]:
for sequence in generated_sequences:
    text = tokenizer.decode(sequence, clean_up_tokenization_spaces=True)
    print(text)
    print("-" * 80)

this royal throne of kings, this sceptred isle, a throne of both the king and the queen and, perhaps, a castle or a castle of the gods, one of those tall five - story levels from which the nose of the mightiest man
--------------------------------------------------------------------------------
this royal throne of kings, this sceptred isle has been diluted in the minds of your communities. 
 now take your sisters and leave the city. call for aid from the one you claim as king. 
 they fled as they heard the voice.
--------------------------------------------------------------------------------
this royal throne of kings, this sceptred isle. be certain to take an interest in his staff - it is hidden in the rubble of the dam. let the asturians and the asturians take a cautious interest in your staff as well. otherwise
--------------------------------------------------------------------------------
this royal throne of kings, this sceptred isle to be filled with bronze that can be caught 